In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import time
from tqdm import tqdm
from tqdm import trange
%matplotlib inline

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from IPython.display import clear_output

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")

In [2]:
start_token = "_"
eos = '#'
MAX_LENGTH = 30

In [3]:
data_pairs = pd.read_csv("base_pairs.csv", index_col=0)
data_pairs.loc[:,"word_form"] = data_pairs.word_form.apply(lambda x: start_token + x + eos)
data_pairs.loc[:,"word_norm"] = data_pairs.word_norm.apply(lambda x: start_token + x + eos)
data_pairs.head()

/Users/viktor/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,lex_id,word_norm,word_form,morph_features_form,morph_features_norm
0,1,_ёж#,_ежа#,"NOUN,anim,masc sing,gent","NOUN,anim,masc sing,nomn"
1,1,_ёж#,_ежу#,"NOUN,anim,masc sing,datv","NOUN,anim,masc sing,nomn"
2,1,_ёж#,_ежа#,"NOUN,anim,masc sing,accs","NOUN,anim,masc sing,nomn"
3,1,_ёж#,_ежом#,"NOUN,anim,masc sing,ablt","NOUN,anim,masc sing,nomn"
4,1,_ёж#,_еже#,"NOUN,anim,masc sing,loct","NOUN,anim,masc sing,nomn"


In [4]:

s = "".join(data_pairs.word_form)
tokens = np.array(list(set(s)))
    
tok2id = dict(zip(tokens, np.arange(tokens.shape[0])))
id2tok = dict(zip(np.arange(tokens.shape[0]), tokens))

print ('There are {} tokens',len(tokens))

print('Vocabular: ' + "".join(tokens))

There are {} tokens 48
Vocabular: 9'2шж’ъиёещбчл5нгср4аыьяу8_птвйюхо6-фм17#кэзц03д


In [120]:
import re
s_f = np.hstack(data_pairs.morph_features_form.apply(lambda x: re.split(",| ", x)).values)
tokens_f = np.array(list(set(s_f)))
    
tok2id_f = dict(zip(tokens_f, np.arange(tokens_f.shape[0])))
# id2tok_f = dict(zip(np.arange(tokens.shape[0]), tokens))

print ('There are {} tokens',len(tokens_f))

print('Vocabular: ' + "".join(tokens_f))

There are {} tokens 100
Vocabular: AnphFixdAproQuesGRND1perV-beMs-fNUMRimpfFimpfemnNOUNintrGNdrHypoDistdatvPREPSurnabltSuprV-iegen23perADJFADJSloc2LitraccsplurNPROGeoxgentVprePRTSCounVERBpssvanimPatrloctactvV-ejINFNAnumAdjxsingOrgnmascCmp2CONJSgtmErroV-oyinclINTJAf-ploc1PrdxPREDNamePltmQualInmxImpepresgen1neutV-enImpxInitADVBV-shInfrtranms-fDmnsV-eyAbbrCOMPvoctArchfutrV-biexclSlngperfindc2perPRCLCollpastnomnPossPrntimprSubxPRTFinan


In [247]:
a = data_pairs.morph_features_form.apply(lambda x: re.split(",| ", x)).values

In [249]:
max_len = 0
for elem in a:
    max_len = max(max_len, len(elem))
print(max_len)

11


In [5]:
SOS_token = tok2id[start_token]
EOS_token = tok2id[eos]

In [6]:
def to_matrix(words, max_len=None, pad=tok2id[eos], dtype=np.int64):
    """Casts a list of words into rnn-digestable matrix"""
    
    max_len = max_len or max(map(len, words))
    words_ix = np.zeros([len(words), max_len], dtype) + pad

    for i in range(len(words)):
        word_ix = list(map(tok2id.get, words[i]))
        words_ix[i, :len(word_ix)] = word_ix

    return words_ix

In [7]:
print('\n'.join(data_pairs.word_form[:10]))
print(to_matrix(data_pairs.word_form[:10]))

_ежа#
_ежу#
_ежа#
_ежом#
_еже#
_ежи#
_ежей#
_ежам#
_ежей#
_ежами#
[[26  9  4 20 40 40 40]
 [26  9  4 24 40 40 40]
 [26  9  4 20 40 40 40]
 [26  9  4 33 37 40 40]
 [26  9  4  9 40 40 40]
 [26  9  4  7 40 40 40]
 [26  9  4  9 30 40 40]
 [26  9  4 20 37 40 40]
 [26  9  4  9 30 40 40]
 [26  9  4 20 37  7 40]]


In [263]:
tok2id_f[eos] = 100

def to_matrix_f(words, max_len=None, pad=tok2id_f[eos], dtype=np.int64):
    """Casts a list of words into rnn-digestable matrix"""
    
    max_len = max_len or max(map(len, words))
    words_ix = np.zeros([11, max_len], dtype) + pad

    for i in range(len(words)):
        word_ix = list(map(tok2id_f.get, words[i]))
        words_ix[i, :len(word_ix)] = word_ix

    return words_ix

In [177]:
to_matrix_f

<function __main__.to_matrix_f>

In [178]:
# Creating training and validation sets using an 80-20 split

input_train, input_test, target_train, target_test = train_test_split(data_pairs.drop("word_form", axis=1), data_pairs.word_form.values, test_size=0.2)

In [179]:
class pairs_Dataset(Dataset):
    def __init__(self, data, data_form):
        self.data = data
        self.target = data_form
    def __len__(self):
        'Denotes the total number of samples'
        return len(self.data)

    def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        return self.data.word_norm.values[index], re.split(",| ", self.data.morph_features_form.values[index]), self.target[index]

In [180]:
BATCH_SIZE = 22

train_data = pairs_Dataset(input_train, target_train)

dataset = DataLoader(train_data, batch_size=BATCH_SIZE, drop_last=True, shuffle=True)

In [264]:
for batch in dataset:
    print(batch)
    print(batch[1])
    print(to_matrix_f(batch[1]))
    break

[('_шпарящийся#', '_примазывавшийся#', '_буферизовался#', '_подкапливаемый#', '_копалов#', '_запихал#', '_калининградгоргаз#', '_надстругивающийся#', '_разляпываюсь#', '_подкапанный#', '_оковывающийся#', '_юниор#', '_развенчиваемый#', '_изожжённый#', '_малогазопроницаемый#', '_упластывающийся#', '_закуролесивший#', '_обкаливший#', '_энергоэффективный#', '_сострогавшийся#', '_абдеррезакович#', '_акобович#'), [('PRTF', 'PRTF', 'VERB', 'PRTF', 'ADJS', 'VERB', 'NOUN', 'PRTF', 'VERB', 'PRTF', 'PRTF', 'NOUN', 'PRTF', 'PRTF', 'ADJF', 'PRTF', 'PRTF', 'PRTF', 'ADJF', 'PRTF', 'NOUN', 'NOUN'), ('impf', 'impf', 'perf', 'impf', 'Qual', 'perf', 'inan', 'impf', 'impf', 'perf', 'impf', 'anim', 'impf', 'perf', 'inan', 'impf', 'perf', 'perf', 'anim', 'perf', 'anim', 'anim'), ('intr', 'intr', 'intr', 'tran', 'plur', 'tran', 'masc', 'intr', 'intr', 'tran', 'intr', 'masc', 'tran', 'tran', 'masc', 'intr', 'intr', 'tran', 'plur', 'intr', 'masc', 'masc')], ('_шпарящихся#', '_примазывавшимся#', '_буферизуюсь#'

In [265]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.batch_size = BATCH_SIZE
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, self.batch_size, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.batch_size, self.hidden_size, device=device)

In [276]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, morph_feat_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.morph_size = morph_feat_size
        self.batch_size = BATCH_SIZE
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.emb_morph = nn.Embedding(self.morph_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 13, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, morph_feat, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, self.batch_size, -1)
        embedded = self.dropout(embedded)
        
        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(1),
                                 encoder_outputs)
        
        embedded_feat = self.emb_morph(morph_feat).view(1, self.batch_size, -1)
        
#         print(morph_feat)
#         print(self.emb_morph(morph_feat).shape)
#         print(embedded_feat.shape)
        output = torch.cat((embedded[0], attn_applied.squeeze(), embedded_feat[0]), 1)
#         print(output.shape)
#         print(self.attn_combine(output).unsqueeze(0))
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, self.batch_size, self.hidden_size, device=device)

In [277]:
teacher_forcing_ratio = 0.5

def train_batch(X, X_f, y, encoder, decoder, encoder_opt, decoder_opt, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_opt.zero_grad()
    decoder_opt.zero_grad()
    max_length = X.shape[1]
    encoder_outputs = torch.zeros(BATCH_SIZE, max_length, encoder.hidden_size, device=device)
    loss = 0
    for i in range(X.shape[1]):
        encoder_output, encoder_hidden = encoder(
            X[:, i], encoder_hidden)
        encoder_outputs[:, i] = encoder_output[0]

    decoder_input = torch.tensor([[int(SOS_token)] * BATCH_SIZE], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if np.random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(y.shape[1]):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, X_f, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, y[:, di])
            decoder_input = y[:, di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(y.shape[1]):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, X_f, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, y[:, di])
#             if decoder_input.item() == EOS_token:
#                 break

    loss.backward()

    encoder_opt.step()
    decoder_opt.step()

    return loss.item()

In [278]:
def plot_history(train_history, title='loss'):
    plt.figure()
    plt.title('{}'.format(title))
    plt.plot(train_history, label='train', zorder=1)    
    plt.xlabel('train steps')
    plt.grid()
    plt.show()
    
    
def train(data, encoder, decoder, n_epochs, print_every=1, plot_every=1, learning_rate=0.01):
    start = time.time()
    plot_losses = []

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    
    criterion = nn.NLLLoss()

    for epoch in range(n_epochs):
        encoder.train()
        decoder.train()
        loss = 0
        for batch_num, batch in zip(trange(len(data)), data):
            max_len = max(max(map(len, batch[0])), max(map(len, batch[2])))
            X = torch.from_numpy(to_matrix(batch[0], max_len=MAX_LENGTH)).to(device)
            X_f = torch.from_numpy(to_matrix_f(batch[1])).to(device)
            y = torch.from_numpy(to_matrix(batch[2], max_len=MAX_LENGTH)).to(device)
            loss = train_batch(X, X_f, y, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
            plot_losses.append(loss / X.shape[0])
        print("Loss - {}".format(loss))
        clear_output()
        plot_history(plot_losses)

In [279]:
hidden_size = 256
encoder1 = EncoderRNN(tokens.shape[0], hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, tokens.shape[0], tokens_f.shape[0] + 1, dropout_p=0.1).to(device)

train(dataset, encoder1, attn_decoder1, 1)



  0%|          | 0/172832 [00:00<?, ?it/s]

  0%|          | 1/172832 [00:00<36:03:31,  1.33it/s]

  0%|          | 2/172832 [00:01<26:02:56,  1.84it/s]

  0%|          | 3/172832 [00:01<22:44:20,  2.11it/s]

  0%|          | 4/172832 [00:01<21:14:55,  2.26it/s]

  0%|          | 5/172832 [00:02<20:59:17,  2.29it/s]

  0%|          | 6/172832 [00:02<20:01:26,  2.40it/s]

  0%|          | 7/172832 [00:02<19:27:01,  2.47it/s]

  0%|          | 8/172832 [00:03<19:15:09,  2.49it/s]

  0%|          | 9/172832 [00:03<18:53:54,  2.54it/s]

  0%|          | 10/172832 [00:03<18:37:12,  2.58it/s]

  0%|          | 11/172832 [00:04<18:22:44,  2.61it/s]

  0%|          | 12/172832 [00:04<18:11:53,  2.64it/s]

  0%|          | 13/172832 [00:04<18:05:00,  2.65it/s]

  0%|          | 14/172832 [00:05<17:56:26,  2.68it/s]

KeyboardInterrupt: 